In [ ]:
import numpy as np
import pandas as pd
from ftag.hdf5 import H5Reader 

In [ ]:
! h5ls -v maya.h5/muons

In [ ]:
! h5ls -v maya.h5/muon_tracks

In [ ]:
fname = "maya.h5"
reader = H5Reader(fname, batch_size=100, jets_name="muons")

muon_variables = ["pt", "eta", "ptvarcone30Rel", "iffClass"]
track_variables = ["pt", "eta", "dr_lepton", "leptonID", "valid"]
data = reader.load({"muons": muon_variables, "muon_tracks": track_variables}, num_jets=3_453) 

In [ ]:
muons = data['muons']
tracks = data['muon_tracks']

In [ ]:
muons

In [ ]:
tracks

In [ ]:
def compute_isolation(muons, tracks, ptcone_size=0.4, n_entries=-1):

    dict_muon_dataframe = {
        "pt": [],
        "eta": [],
        "isolation": [],
        "iffClass": [],
    }
    
    # here we loop over muons
    loop_counter = 0
    for muon, track_array in zip(muons, tracks):
        muon_pt = muon[0]
        muon_eta = muon[1]
        muon_ptvarcone30Rel = muon[2]
        muon_class = muon[3]
        
        sum_ptcone = 0.0
        # here we loop over tracks associated with muons
        for track in track_array:
            if not track[0]: continue
            track_pt = track[1]
            track_eta = track[2]
            track_dR_to_lepton = track[3]
            track_is_muon = (abs(track[4]) == 13)

            # compute track-based isolation
            if not track_is_muon and (track_dR_to_lepton < ptcone_size):
                sum_ptcone += track_pt

        relative_isolation = sum_ptcone / muon_pt
        loop_counter += 1
        if loop_counter > n_entries and n_entries > 0: break
            
        print(f"ptcone{int(ptcone_size*100)}/pt: {relative_isolation}, muon_ptvarcone30Rel: {muon_ptvarcone30Rel} , isolation class: {muon_class}")
        dict_muon_dataframe["pt"].append(muon_pt)
        dict_muon_dataframe["eta"].append(muon_eta)
        dict_muon_dataframe["isolation"].append(relative_isolation)
        dict_muon_dataframe["iffClass"].append(muon_class)

    return dict_muon_dataframe

dict_muons = compute_isolation(muons, tracks, 0.3, 10)

# convert this to a dataframe